In [13]:
import pandas as pd

In [42]:
def flattenColumn(input, column):
    column_flat = pd.DataFrame([[i, c_flattened] for i, y in input[column].apply(list).iteritems() for c_flattened in y], columns=['I', column])
    column_flat = column_flat.set_index('I')
    return input.drop(column, 1).merge(column_flat, left_index=True, right_index=True)

### Build rent prices data set [2009-2017]

In [43]:
rent_prices = pd.read_json("./data/renting_prices.json")
plz_to_district = pd.read_json("./data/berlin_plz.json", orient="records")

plz_to_district = plz_to_district.T.reset_index().rename(columns={'index': 'district',0: 'post_code'})
plz_to_district = flattenColumn(plz_to_district, 'post_code')

In [44]:
rent_prices = rent_prices.merge(plz_to_district, left_on="zip", right_on="post_code").drop(['zip','post_code'], 
                                                                                               axis=1)
rent_prices = rent_prices.groupby(['district','year'], as_index=False).mean()

In [45]:
rent_prices.to_csv('./data/rent_prices_until_15.csv', index=False, encoding="utf-8")

In [6]:
rent_prices = pd.read_csv("./data/rent_prices_until_17.csv",sep="\t")

In [8]:
rent_prices.head(2)

,district,year,rent_cold
0,Charlottenburg-Wilmersdorf,2009,6.994545
1,Charlottenburg-Wilmersdorf,2010,7.370606


### Build real state transactions dataset

In [64]:
imm_trans = pd.read_csv("./data/wohnimmobilien_transaktion.csv")

In [83]:
imm_trans.year.unique()

array([2013, 2014, 2015, 2016, 2017])

In [72]:
imm_trans = imm_trans.sort_values(by="year")

In [66]:
names = list(imm_trans['Käufer'].unique())
add = list(imm_trans['Verkäufer'].unique())

In [67]:
names.extend(x for x in add if x not in names)

In [69]:
len(names)

78

In [79]:
mill_buy = imm_trans.loc[:,('Käufer','Kaufpreis in Mio. €, ca.')].groupby(['Käufer'], as_index=False).sum()
n_flat_buy = imm_trans.loc[:,('Käufer','Anzahl der Wohnungen, ca.')].groupby(['Käufer'], as_index=False).sum()
mill_sell = imm_trans.loc[:,('Verkäufer','Kaufpreis in Mio. €, ca.')].groupby(['Verkäufer'], as_index=False).sum()
n_flat_sell = imm_trans.loc[:,('Verkäufer','Anzahl der Wohnungen, ca.')].groupby(['Verkäufer'], as_index=False).sum()

In [82]:
print(mill_buy['Kaufpreis in Mio. €, ca.'].sum(), n_flat_buy['Anzahl der Wohnungen, ca.'].sum(),
      mill_sell['Kaufpreis in Mio. €, ca.'].sum(), n_flat_sell['Anzahl der Wohnungen, ca.'].sum())

10776.5 55552.949 10834.5 55717.94899999999
